<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/Key2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!git clone https://github.com/MarkSecada/key2vec.git

Cloning into 'key2vec'...
remote: Enumerating objects: 75, done.
remote: Total 75 (delta 0), reused 0 (delta 0), pack-reused 75
Unpacking objects: 100% (75/75), done.


In [ ]:

!git clone https://github.com/uclnlp/inferbeddings/blob/master/data/glove/glove.6B.50d.txt.gz

Cloning into 'glove.6B.50d.txt.gz'...
fatal: repository 'https://github.com/uclnlp/inferbeddings/blob/master/data/glove/glove.6B.50d.txt.gz/' not found


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:

%%time
!python /content/key2vec/test.py

Streaming output truncated to the last 5000 lines.
4994 20 280
4993 20 243
4992 20 222
4991 20 235
4990 20 285
4989 20 323
4988 20 346
4987 20 297
4986 20 281
4985 20 267
4984 20 285
4983 20 301
4982 20 302
4981 20 229
4980 20 213
4979 20 301
4978 20 199
4977 20 299
4976 20 309
4975 20 273
4974 20 192
4973 20 323
4972 20 308
4971 20 298
4970 20 269
4969 20 263
4968 20 287
4967 20 212
4966 20 310
4965 20 288
4964 20 229
4963 20 263
4962 20 312
4961 20 288
4960 20 299
4959 20 339
4958 20 303
4957 20 335
4956 20 311
4955 20 285
4954 20 229
4953 20 330
4952 20 244
4951 20 322
4950 20 318
4949 20 232
4948 20 289
4947 20 288
4946 20 387
4945 20 309
4944 20 268
4943 20 251
4942 20 249
4941 20 355
4940 20 239
4939 20 289
4938 20 299
4937 20 271
4936 20 231
4935 20 320
4934 20 288
4933 20 307
4932 20 320
4931 20 242
4930 20 288
4929 20 275
4928 20 358
4927 20 274
4926 20 290
4925 20 258
4924 20 308
4923 20 297
4922 20 306
4921 20 266
4920 20 304
4919 20 270
4918 20 295
4917 20 335
4916 20 303
4

#test.py

In [ ]:
import key2vec
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import exists

path = '/content/drive/MyDrive/PaperA3/glove.6B.50d.txt'
glove = key2vec.glove.Glove(path)

df =pd.read_csv('/content/drive/MyDrive/PaperA3/Corpus/corpus_BOC_NegCUI.csv') 
df.drop('Unnamed: 0', axis=1, inplace=True) 
df.dropna()

dd=pd.read_csv('/content/drive/MyDrive/PaperA3/Diagnoses_labels/Dictionary.csv')
dictionary = dict(zip(dd.first_cuid, dd.canonical_name))

import ast
def complex_function( x):
    res =x.split(",")
    tms_text=''    
    for i in range(len(res)):
        try:  
            if(res[i].upper().startswith("NOT")):
              tms_text=tms_text +'NOT '+dictionary[res[i][4:].upper()] +', '
            else:
              tms_text=tms_text +dictionary[res[i].upper()] +', '  
        except :
              print(res[i])    
    return tms_text 

df=df.dropna()
df['Text'] = df.apply(lambda x: complex_function(x['term']), axis=1)
df.shape

df1=df[['label','Text']]
df1.head()
df1.shape

df1.label=df1.label.astype(float).astype(int)


df_remain= pd.DataFrame(columns = ['label','Text'])
for i in range(1,30):
  filename="/content/drive/MyDrive/PaperA3/Key2Vec/Key2Vec_20_"+str(i)+".csv"
  
  if (exists(filename)==True):
          
          df2=pd.read_csv(filename)
          df2=df2[['label','Text']]
          print(filename,len(df2))
          df_remain=pd.concat([df_remain, df2])
ls_remain=df_remain.label.astype(float).astype(int).unique().tolist() 

df3=df1[~df1.label.isin(ls_remain)]
print(df3.shape)


data_all_new= []
for j in range(len(df3)):
    try:
        m = key2vec.key2vec.Key2Vec(df3.iloc[j].Text, glove)
        m.extract_candidates()
        m.set_theme_weights()
        m.build_candidate_graph()
        ranked = m.page_rank_candidates()
        ls=[]
        for row in ranked:
            ls.append( row.text)
           
        str_tmp = ', '.join([str(elem) for elem in ls])
        data_all_new.append( [df3.iloc[j].label,str_tmp])
        print(len(df3)-j,len(ls),len(str_tmp)) 
        if (j % 1000 ==0):
              filename=int(j/1000+20)
              pd.DataFrame(data_all_new,columns=['label','Text']).to_csv('/content/drive/MyDrive/PaperA3/Key2Vec/Key2Vec_20_'+str(filename)+'.csv')
              data_all_new= []
    except:
      print('Error')          
pd.DataFrame(data_all_new,columns=['label','Text']).to_csv('/content/drive/MyDrive/PaperA3/Key2Vec/Key2Vec_20_'+str(filename)+'.csv')